In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import loading

In [2]:
def selu(x):
    """
    Selu activation function
    see: https://arxiv.org/pdf/1706.02515.pdf
    """
    with ops.name_scope('elu'):
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
    return scale * tf.where(x >= 0.0, x, alpha * tf.nn.elu(x))

In [3]:
input_layer = tf.placeholder(dtype=tf.float32, shape=(None, 2, 4, 9), name='input')
conv = tf.layers.conv2d(
    input_layer, 
    filters=16, kernel_size=(4, 4), 
    activation=selu, data_format='channels_first', 
    name='conv1'
)
dense = tf.layers.dense(conv, 36, activation=selu, name='dense')
filtered = dense * tf.reshape(tf.reduce_sum(input_layer, axis=1), (-1, 36))
filtered = filtered / tf.reduce_sum(filtered)
output = tf.nn.softmax(filtered)

In [7]:
variables = tf.global_variables()
[v for v in variables if 'conv1/kernel' in v.name]

[<tf.Variable 'conv1/kernel:0' shape=(4, 4, 2, 16) dtype=float32_ref>]

In [2]:
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir_ = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
fake_data = loading.default_loader(os.path.join(datadir, 'fake news (with groups).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))

df = hvhdata[0]
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])